In [2]:
import pandas as pd
import csv
import os 
import datetime as dt
import teradatasql as td
from selenium import webdriver
from selenium.webdriver.common.by import By
import datetime
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from pathlib import Path

In [3]:
print(os.listdir('C:/Users/Chweber/Downloads'))
print(os.getcwd())

['desktop.ini', '~$All Leads - Measurement 10-24-2024 3-05-41 PM.xlsx', '~$High Value New Gamer - New to MGM.pptx', '~$multiprop_spend_data.xlsx']
C:\Users\Chweber\Host NBA CRM Stuff\Host NBA Pipeline


<h1> Host NBA CRM Data Pipeline </h1>

<h3>Data Pulling</h3>

In [6]:
host_dynamics_urls = ['https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=5a944623-fe03-ee11-8f6e-6045bd006149&viewType=4230&lid=1722024607660'
                      , 'https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=356ac88c-fc03-ee11-8f6e-6045bd006793&viewType=4230'
                      , 'https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=feb7d7bc-fc03-ee11-8f6e-6045bd006793&viewType=4230']

with open('asdf.txt','r') as file2:
    pw = file2.read()
    
# download_dir = 'C:/Users/Chweber/Host NBA CRM Stuff/CRM Files'

chrome_options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=chrome_options)
driver.get(host_dynamics_urls[0])
# print(host_dynamics_urls[0])
input_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input.form-control.ltr_override.input.ext-input.text-box.ext-text-box'))
)
# text_entry = driver.find_element(By.CSS_SELECTOR, 'input.form-control.ltr_override.input.ext-input.text-box.ext-text-box')
input_element.send_keys('cweber@mgmresorts.com')
input_element.send_keys(Keys.ENTER)

input_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'input28'))
)
input_element.send_keys('604591')
input_element.send_keys(Keys.ENTER)

input_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'input54'))
)
input_element.send_keys(pw)
input_element.send_keys(Keys.ENTER)

files = os.listdir('C:/Users/Chweber/Downloads')
dwnld_length = len(files)
set = dwnld_length

n=1
try:
    while dwnld_length != set+3:
        timeout = 1000
    
        # waits until button loads to click
        button = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.ID, 'activitypointer|NoRelationship|HomePageGrid|Mscrm.HomepageGrid.activitypointer.ExportToExcel.Menu$button0'))
        )
        button.click()
        print('button clicked successfully')
    
        # gets length of the current downloads directory
        files = os.listdir('C:/Users/Chweber/Downloads')
        set2 = len(files)
    
        # until the file has finished downloading, keep this loop going
        while dwnld_length != set2+1:
            print('bedtime')
            time.sleep(60)
            files = os.listdir('C:/Users/Chweber/Downloads')
            dwnld_length = len(files)
    
        # get next download, caps it at three for each of the host nba markets
        # while n < 3:
        driver.get(host_dynamics_urls[n])
        n+=1
        time.sleep(20)
except:
    print('All files downloaded')


button clicked successfully
bedtime
bedtime
button clicked successfully
bedtime
button clicked successfully
bedtime
All files downloaded


In [7]:
# gets files from the directory
files = os.listdir('C:/Users/Chweber/Downloads')

# specifically the three nba files just downloaded
files = files[:3]

for i in files:
    # reads each excel file
    excel_file = 'C:/Users/Chweber/Downloads/'+i
    # puts file into a dataframe
    df = pd.read_excel(excel_file, engine='openpyxl')
    # df.to_csv('Host NBA Pipeline/CRM Files/'+i[:-5]+'.csv')
    df.to_csv('CRM Files/'+i[:-5]+'.csv')

C:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<h3>Data Cleansing/Validation</h3>

In [9]:
# file_path = os.listdir('Host NBA Pipeline/CRM Files')
file_path = os.listdir('CRM Files')
print(file_path)
# applies data transformations necessary for SQL queries
for i in file_path[:3]:
    # df = pd.read_csv('Host NBA Pipeline/CRM Files/'+i)
    df = pd.read_csv('CRM Files/'+i)
    # removes unwanted columns
    df = df.drop(columns=['(Do Not Modify) Task','(Do Not Modify) Row Checksum','(Do Not Modify) Modified On','Unnamed: 0', 'Task Completion Comments'])

    # creates Dateloaded column, formats as appropriate for a timestamp value
    # df['Dateloaded'] = pd.Timestamp.now().floor('s')
    # df['Dateloaded'] = df['Dateloaded'].astype(str)
    # df['Dateloaded'] = pd.to_datetime(df['Dateloaded'].str.replace(r'[^\d\-: ]', '', regex=True), errors='coerce').dt.normalize()
    # df['Dateloaded'] = df['Dateloaded'].dt.strftime('%m-%d-%Y %H:%M:%S')

    # converts the columns representing dates to the datetime format
    # df['Created On'] = pd.to_datetime(df['Created On'].str.replace(r'[^\d\-: ]', '', regex=True),format='%Y-%m-%d', errors='coerce').dt.normalize()
    # df['Due Date'] = pd.to_datetime(df['Due Date'].str.replace(r'[^\d\-: ]', '', regex=True),format='%Y-%m-%d', errors='coerce').dt.normalize()
    # df['Actual End'] = pd.to_datetime(df['Actual End'].str.replace(r'[^\d\-: ]', '', regex=True),format='%Y-%m-%d', errors='coerce').dt.normalize()
    
    df['Created On'] = pd.to_datetime(df['Created On'].str.replace(r'[^\d\-: ]', '', regex=True), errors='raise')
    df['Due Date'] = pd.to_datetime(df['Due Date'].str.replace(r'[^\d\-: ]', '', regex=True), errors='raise')
    df['Actual End'] = pd.to_datetime(df['Actual End'].str.replace(r'[^\d\-: ]', '', regex=True), errors='raise')
    df['Actual End'] = df['Actual End'].fillna(pd.Timestamp.today().normalize()).dt.date


    df['Created On'] = pd.to_datetime(df['Created On']).dt.date
    df['Due Date'] = pd.to_datetime(df['Due Date']).dt.date
    df['Actual End'] = pd.to_datetime(df['Actual End']).dt.date
    
    # Normalize (strip time information) if datetime parsing is successful
    # df['Created On'] = df['Created On'].dt.normalize()
    # df['Due Date'] = df['Due Date'].dt.normalize()
    # df['Actual End'] = df['Actual End'].dt.normalize()

    
    # removes all whitespace, necessary for SQL
    df = df.rename(columns={'Created On':'Created_On','Host ID (Owning User) (User)':'Host_ID','User Name (Owning User) (User)':'User_Name','Property (Owning User) (User)':'Property','MGM Rewards # (Regarding) (Contact)':'MGM_Rewards_ID','Due Date':'Due_Date','Actual End':'Actual_End','Activity Status':'Activity_Status','Channel Outcome':'Channel_Outcome','NBA ID':'NBA_ID','NBA Type':'NBA_Type'})
    
    # removes duplicate NBA tasks
    df = df.drop_duplicates()

    # fills all na with 0
    df = df.fillna(0)

    # ensures data types for each column are appropriate
    df['Created_On'] = df['Created_On'].astype(str)
    df['Owner'] = df['Owner'].astype(str)
    df['Host_ID'] = df['Host_ID'].astype(int)
    df['User_Name'] = df['User_Name'].astype(str)
    df['Property'] = df['Property'].astype(str)
    df['Subject'] = df['Subject'].astype(str)
    df['MGM_Rewards_ID'] = df['MGM_Rewards_ID'].astype(int)
    df['Regarding'] = df['Regarding'].astype(str)
    df['ADW'] = df['ADW'].astype(int)
    df['Due_Date'] = df['Due_Date'].astype(str)
    df['Actual_End'] = df['Actual_End'].astype(str)
    df['Activity_Status'] = df['Activity_Status'].astype(str)
    df['Channel'] = df['Channel'].astype(str)
    df['Channel_Outcome'] = df['Channel_Outcome'].astype(str)
    df['NBA_ID'] = df['NBA_ID'].astype(int)
    df['NBA_Type'] = df['NBA_Type'].astype(str)

    # uses list comprehension to add Dateloaded as the first column
    # columns = ['Dateloaded'] + [col for col in df.columns if col != 'Dateloaded']
    columns = [col for col in df.columns if (col != 'Dateloaded' and col != 'ATT (Corp)')]
    
    # specifies order of columns
    df = df[columns]
  
    # print(df['Created_On'].unique())  
    # print(df['Actual_End'].unique())  
    # print(df['Due_Date'].unique())  
    # creates prepped versions of the dataframes, tab as the delimiter which is necessary for teradata sql import
    # df.to_csv('Host NBA Pipeline/Prepped CRM Files/prepped_'+i[:-4]+'.txt', sep = '\t', index=False)
    df.to_csv('Prepped CRM Files/prepped_'+i[:-4]+'.txt', sep = '\t', index=False)

['#CM. Smart Player Outreach (NBA) - Las Vegas 1-11-2025 12-54-01 PM.csv', '#CM. Smart Player Outreach (NBA) - Las Vegas 1-13-2025 4-38-16 PM.csv', '#CM. Smart Player Outreach (NBA) - PCOMM 1-11-2025 12-55-26 PM.csv', '#CM. Smart Player Outreach (NBA) - PCOMM 1-13-2025 4-39-25 PM.csv', '#CM. Smart Player Outreach (NBA) - Regionals 1-11-2025 12-56-12 PM.csv', '#CM. Smart Player Outreach (NBA) - Regionals 1-13-2025 4-40-25 PM.csv', '.ipynb_checkpoints']


C:\Users\Chweber\AppData\Local\Temp\ipykernel_35696\1200929538.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Created On'] = pd.to_datetime(df['Created On'].str.replace(r'[^\d\-: ]', '', regex=True), errors='raise')


<h3>Teradata Driver Integration</h3>

In [ ]:
# specifies names of the tables 
table_names = ['ae_smart_player_outreach_lv','ae_smart_player_outreach_pcomm','ae_smart_player_outreach_reg']

# grabs location of prepped files
prepped_files = os.listdir('Prepped CRM Files')

# grabs location of sql queries
queries = os.listdir('SQL Queries')

# purpose of the function is to create/refresh the tables
def table_creation(cur,n):
    print('Creating table...')
    try:
        cur.execute("""
                    drop table prd_mktg_opr_tb."""+table_names[n]
                    )
    except:
        print('table not found')
        
    cur.execute ("""
                create table prd_mktg_opr_tb."""+table_names[n]+""" (
                    Dateloaded TIMESTAMP,
                    Created_On varchar(255),
                    "Owner" varchar(255),
                    Host_ID integer,
                    User_Name varchar(255),
                    Property varchar(255),
                    Subject varchar(255),
                    MGM_Rewards_ID integer,
                    Regarding varchar(255),
                    ADW integer,
                    Due_Date varchar(255),
                    Actual_End varchar(255),
                    Activity_Status varchar(255),
                    Channel varchar(255),
                    Channel_Outcome varchar(255),
                    NBA_ID integer,
                    NBA_Type varchar(255)
                    );
                  """)
    print('Table creation successful for '+table_names[n])

# purpose of the function is to import the prepped files into the newly created tables
def import_to_teradata(df,cur):
    try:
        # necessary for data import, list out columns and then ? as the wildcard
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?' for i in range(len(df.columns))])
        # print(columns+'\n'+placeholders)
        
        print('Inserting values...')
        # sql statement
        sql = f"INSERT INTO prd_mktg_opr_tb.{table_names[n]} ({columns}) VALUES ({placeholders})"

        # rows
        data_to_insert = [tuple(row) for row in df.values]

        # directs cursor to import the data 
        cur.executemany(sql, data_to_insert)

    except Exception as e:
        print(f'An error occurred: {e}')
        try:
            cur.execute("SELECT {fn teradata_get_errors()} AS error_details")
            error_details = cursor.fetchone()
            print("Error Details:", error_details)
            
            cur.execute("SELECT {fn teradata_nativesql()} AS native_sql_error")
            native_sql_error = cursor.fetchone()
            print("Native SQL Error:", native_sql_error)

        except Exception as inner_e:
            print(f"An error occurred while retrieving error details: {inner_e}")
            
    finally:
        print("Data import successful")
    
def execute_query(cur,lines): 
    # breaks apart each query into its individual statements
    statements = lines.split(';')

    # removes whitespace
    statements = [i.strip() for i in statements]
    # statements = [i.replace('\n', '').replace('\t', '') for i in statements]

    # grab the sql statements which involve volatile table creation, select statements, etc.
    keywords = ["create", "CREATE", "sel", "SELECT",'insert','INSERT','table']
    
    # only grab the statements which have a match from the list above
    statements = [i for i in statements if any(keyword in i for keyword in keywords)]

    print('Executing query...')

    # iterate over the statements, executing them with the cursor
    for i in statements:
        print('\t'+i[:30])
        cur.execute(i)
    print('Query executed')

# grabs password
with open('asdf.txt','r') as file2:
    pw = file2.read()

# for each table which needs refreshing
for n in range(3):
    # script_dir = os.path.dirname(os.path.abspath(Path.cwd()))
    
    # getting the absolute path to your prepped file and queries file
    prepped_file_path = os.path.join('Prepped CRM Files/',prepped_files[n+1])
    queries_file_path = os.path.join('SQL Queries/',queries[n+1])
    
    # prepped_file_path = 'CRM Files/'+prepped_files[n+1]
    # reading each CSV file and queries file using the absolute path
    df = pd.read_csv(prepped_file_path,sep='\t')

    # connects to Teradata server, specifies host, logmech, and user to execute queries
    with td.connect(host='TDPROD',logmech='LDAP',user='chweber',password=pw) as con:
        with con.cursor () as cur: 
            # create table
            table_creation(cur,n)
            # import prepped data to these tables
            import_to_teradata(df,cur)
            # reads query files, passes it to the execute function
            with open(queries_file_path,'r') as file:
                lines = file.read()
            # execute queries, 40+ statements in each
            execute_query(cur,lines)

Creating table...
Table creation successful for ae_smart_player_outreach_lv
Inserting values...
Data import successful
Executing query...
	create volatile table step1 as
	/*sel * from step1 where prope
	-- select * from step1a where 
	/*************  Add in BetMGM 
	--select * from step1bb
 
crea
	-- select * from step1c


/***
	create volatile table step1d a
	--drop table step1g
create vol
	--sel * from step1e where trip
	--sel * from step1f where trip
	--select * from step1g
/******
	--select * from step2

/******
	--select * from step3 sample 1
	--drop table step4
create vola
	--sel * from step4
-- select *
	/*********************  Add in
	-- select * from step5b order 
	-- select * from step5bb order
	-- select * from step5c order 
	-- select * from step5d order 
	/****************** Create fil
	-- select * from step1g where 
	-- select * from step7 order b
	--select * from step8 order by
	/************** Summarized Vie
	/************ Still need to up
	create volatile table step10 

<h4>All that's left is to refresh the Power BI dashboards!</h4>